<a href="https://colab.research.google.com/github/Yolantele/ML-data-clasifier/blob/master/tabular_neural_net_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular Model For Waste Data Classification


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.
df - data_frame

In [0]:
# mount to google drive and load files
from google.colab import drive
drive.mount('/content/drive')


In [4]:
# check file is loaded OK
path = '/content/drive/My Drive/data/'
print(path)

/content/drive/My Drive/data/


In [77]:
# load train data frame
# todo: 1. iterate through nl data and export only with material, eural code fields
#  then translate to en. this will increase train and test data sets and include missing material categories that appear in validation set
train_file = '/enMaterialData.csv'
df = pd.read_csv(path + train_file)



# load test data frame with material field empty
test_df = pd.read_csv(path + '/enWithoutMaterialData.csv')

df.head()

,reason,origin,color,state,size,consistency,otherCode,material4,material3,material2,material,mType,composite2,composite1,cType,indirectProduct,directProduct,pType,mixedOrPure,cleanOrDirty,euralDescription,euralCode,description,/0
0,NaN,NaN,NaN,NaN,NaN,slurry,NaN,NaN,NaN,oil,organic material,Soy,NaN,dry,NaN,dry,NaN,NaN,1.0,NaN,material unsuitable for consumption or processing,20304,Soyadroes technically,NaN
1,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,wood,branches,NaN,NaN,NaN,wood,NaN,NaN,0.0,NaN,waste from forestry,20107,Branches,NaN
2,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,cocoa,caps,NaN,NaN,NaN,cocoa,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,Cocoa shells,NaN
3,NaN,NaN,NaN,debris,NaN,vast,NaN,NaN,NaN,NaN,sand,NaN,NaN,debris,NaN,debris,NaN,NaN,1.0,1.0,"mixtures of concrete, stones, tiles or ceramic...",170107,Debris with Sand,NaN
4,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,organic material,Soy,NaN,NaN,NaN,organic material,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,soya,NaN


#cats and conts 

**categorical variables - cat_names:**

what you're using to make predictions with.  (description, euralDescription, composite1, mType).
for categorical variables - will use Embeddings 

**continuos variables - cont_names:**

any catgeory that has limited number of possible (integer) choises: true/false ; fields such as cleanOrContaminated or mixedOrPure
continuos varibale names - they are sent to Neural net as pixels 

**uses processor - procs:**

a number of processes that deal with data in some ways: FillMissing, Categorify, Normalize both train and test dtata sets

In [0]:
# dependable variable field:
dep_var = 'material'
 
cat_names = ['euralDescription', 'description'] # limited categorical variables 
cont_names = ['euralCode'] # limited option numeric or true/false values sent to NN as 'pixels'
# cont_names = ['mixedOrPure', 'cleanOrDirty']
# processors:
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[420:2100].copy(), path=path + train_file, cat_names=cat_names, cont_names=cont_names)

Next:
data block api
add labels - tell which cols - column it is 

and get back data bunch
split train and test data ranges 800,1000


In [80]:
data = (TabularList.from_df(df, path=path+ train_file, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(950,1200)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:537: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
PCFIC, non-iron, flint, zeolite, syrup...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [81]:
data.show_batch(rows=10)

euralDescription,description,euralCode,target
wood,"A wood, external",0.1376,wood
soil and stones not covered by 17 05 03,"Land, Fletcher Hotel Marijke, Dorpsstraat 23, Bergen",0.1433,ground
biodegradable waste,Composting green waste earoob,0.7011,organic material
soil and stones not covered by 17 05 03,ancestors,0.1433,sand
soil and stones not covered by 17 05 03,SOIL ORGANICALLY CONTAMINATED WET,0.1433,ground
concrete,Concrete rubble from various places in North Holland,0.1357,grind
biodegradable waste,Organic material,0.7011,organic material
soil and stones not covered by 17 05 03,"Land, Bouwbakkie, Kwikstaartlaan 38, Alkmaar",0.1433,ground
soil and stones not covered by 17 05 03,"SOIL, CONTAMINATED",0.1433,ground
material unsuitable for consumption or processing,Zaden/Renewi-Takii Europe,-2.6781,organic material


In [82]:
print(len(data.classes), 'categories for material --->', data.classes, )

95 categories for material ---> ['AVI-bodemas', 'Cork', 'EPS', 'Glass', 'HDPE', 'LDPE', 'ON', 'PA', 'PC', 'PET', 'PFOS', 'PMMA', 'POM', 'PP', 'PS', 'PVC', 'Perlite', 'RIGHT NOW', 'RVS', 'SECTION', 'Soy', 'UP', 'activated carbon', 'aluminum', 'asbestos', 'beer', 'believe', 'bentonite', 'beryllium', 'bitumen', 'bold', 'bread', 'carton', 'cast', 'cement', 'ceramic', 'ceramic material', 'cocoa', 'collect', 'compost', 'dill', 'dough', 'eps', 'fatty acids', 'fly ash', 'foam rubber', 'glass wool', 'glycerine', 'grind', 'ground', 'insulation material', 'iron', 'know', 'kraftkarton', 'kwik', 'lavaliet', 'lime', 'lithium', 'magnesium', 'melamine', 'messing', 'metal', 'minerals', 'molasses', 'most', 'nickel', 'nylon', 'oil', 'optical fiber', 'organic material', 'paper', 'plastic', 'polyester', 'porcelain', 'pvc', 'resin', 'rock wool', 'rubber', 'rvs', 'sand', 'sintels', 'snail wool', 'snails', 'starch', 'steel', 'stones', 'stories', 'sugar', 'teer', 'turf', 'tyleen', 'water', 'wine', 'wood', 'zin

In [83]:
print(data)

TabularDataBunch;

Train: LabelList (1870 items)
x: TabularList
euralDescription material unsuitable for consumption or processing; description Soyadroes technically; euralCode -2.6781; ,euralDescription waste from forestry; description Branches; euralCode -2.6818; ,euralDescription material unsuitable for consumption or processing; description Cocoa shells; euralCode -2.6781; ,euralDescription mixtures of concrete, stones, tiles or ceramic products other than those mentioned in 17 01 06; description Debris with Sand; euralCode 0.1358; ,euralDescription material unsuitable for consumption or processing; description soya; euralCode -2.6781; 
y: CategoryList
organic material,wood,cocoa,sand,organic material
Path: /content/drive/My Drive/data/enMaterialData.csv;

Valid: LabelList (244 items)
x: TabularList
euralDescription soil and stones not covered by 17 05 03; description #na#; euralCode 0.1433; ,euralDescription biodegradable kitchen and canteen waste; description #na#; euralCode 0.69

pass in data to tabular learner:


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [85]:
learn.fit(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.117789,2.461911,0.299180,00:00
1,1.375463,1.317703,0.680328,00:00
2,0.892217,1.468174,0.684426,00:00
3,0.561458,1.532098,0.704918,00:00


In [72]:
# see into the model
# learn.model.eval()
learn.summary()


TabularModel
Layer (type)         Output Shape         Param #    Trainable 
Embedding            [16]                 960        True      
______________________________________________________________________
Embedding            [146]                462,674    True      
______________________________________________________________________
Dropout              [162]                0          False     
______________________________________________________________________
BatchNorm1d          [1]                  2          True      
______________________________________________________________________
Linear               [200]                32,800     True      
______________________________________________________________________
ReLU                 [200]                0          False     
______________________________________________________________________
BatchNorm1d          [200]                400        True      
_________________________________________________

In [0]:
learn.model_dir = '/content/drive/My Drive/data/models'
learn.save('material_classifier_model')

## Inference  
- run test data set to predict material
- extract the prediction confindece score  

In [106]:
row = 544

data_frame = df
# data_frame = test_df

print('given material -->', data_frame.iloc[row].material)
# print('description ---> ', data_frame.iloc[row].description, ', ', data_frame.iloc[row].euralDescription)
print('description ---> ', data_frame.iloc[row].euralDescription)

given material --> ground
description --->  soil and stones not covered by 17 05 03


In [107]:
#one_row = df.iloc[row]
one_row = test_df.iloc[row]

predicted_cat, total_cat, weights = learn.predict(one_row)
print('predicted material ----> ', predicted_cat)

predicted material ---->  ground


Category Probability - Prediction Confidence score

In [108]:
cat, idx, confidence = learn.predict(one_row)
cat_prob = confidence[idx.item()].item()
print(' category probability --------->', cat_prob)
print(confidence.sum())

 category probability ---------> 0.404806911945343
tensor(1.0000)
